<a href="https://colab.research.google.com/github/heriyang9000/test2/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df01 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df02 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df03 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df04 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df05 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df06 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df07 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df08 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df09 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df10 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df11 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df12 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df13 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df14 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df15 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df16 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df17 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df18 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df19 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df20 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df21 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df22 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df23 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df24 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df25 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df26 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df27 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df28 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df29 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')
df30 = pd.read_csv('https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/AAPL.csv')

print(df01.to_string()) 